# A Media Selection Problem
This notebook is based on a problem posed in Chapter 9 of the AIMMS documentation:<br>
https://documentation.aimms.com/_downloads/AIMMS_modeling.pdf<br>
The media selection problem is an instance of a family of problems called set covering problems. Consider a set $S=\{s_{1}, s_{2}, ..., s_{n}\}$ and a set of subsets of $S$ called $U$. Each element of $U$ has an amount of cost associated with it. The goal is to create a combination of elements of $U$, in a way that each element of $S$ is represented in the resulting combination, and the combination incurs the least amount of cost.

In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

**Problem Description:**<br>
A company is thinking about how to divide its advertising budget among different outlets. There are different groups of audiences that the company is trying to target. Each target group can be reached by a few specific advertising mediums, while there is no single medium that can reach all groups. The company wants to minimize its advetising budget while reaching all of the audience groups.<br>
There are 6 different groups of audiences and 8 different types of advertising media. The following table shows what audience groups each medium can reach, as well as the cost associated with the medium.

In [2]:
reach_df = pd.DataFrame()
reach_df["media"] = ["Glossy magazine", "TV late night", "TV prime time", "Billboard train", "Billboard bus", \
                  "National paper", "Financial paper", "Regional paper"]
reach_df["Audience type 1"] = [1, 0, 0, 1, 0, 0, 0, 1]
reach_df["Audience type 2"] = [0, 1, 1, 0, 0, 0, 1, 0]
reach_df["Audience type 3"] = [0, 1, 0, 0, 1, 0, 0, 0]
reach_df["Audience type 4"] = [1, 0, 0, 0, 0, 1, 0, 0]
reach_df["Audience type 5"] = [0, 0, 0, 0, 0, 0, 1, 1]
reach_df["Audience type 6"] = [0, 0, 1, 1, 0, 1, 0, 0]
reach_df["costs"] = [20000, 50000, 60000, 45000, 30000, 55000, 60000, 52500]

reach_df

,media,Audience type 1,Audience type 2,Audience type 3,Audience type 4,Audience type 5,Audience type 6,costs
0,Glossy magazine,1,0,0,1,0,0,20000
1,TV late night,0,1,1,0,0,0,50000
2,TV prime time,0,1,0,0,0,1,60000
3,Billboard train,1,0,0,0,0,1,45000
4,Billboard bus,0,0,1,0,0,0,30000
5,National paper,0,0,0,1,0,1,55000
6,Financial paper,0,1,0,0,1,0,60000
7,Regional paper,1,0,0,0,1,0,52500


Given the information above, the optimization problem can be summarized as follows:<br>
**Minimiza:**$\;\;\;\;\;\;$total campaign costs,<br>
**Subject to:**$\;\;\;\;\;\;$each audience group needs to be reached at least once<br><br>
We can use the following notation for the mathematical representation of the problem:<br>
$\;\;\;\;\;\;$t$\;\;\;\;\;\;\;\;\;\;$target audiences<br>
$\;\;\;\;\;\;$m$\;\;\;\;\;\;\;\;\;$advertising media<br>
$\;\;\;\;\;\;$$N_{tm}$$\;\;\;\;\;\;$audience t is covered by medium m<br>
$\;\;\;\;\;\;$$c_{m}$$\;\;\;\;\;\;\;\;$cost of medium m<br>
$\;\;\;\;\;\;$$x_{m}$$\;\;\;\;\;\;\;\;$whether medium m is selected<br>
So the Pyomo optimization will look as follows:<br>
**Minimize:**
$$
\underset{}{min} \sum_{m}^{} c_{m}x_{m}
$$
**Subject to:**
$$
\sum_{m}^{}N_{tm}x_{m} \ge 1 \;\;\;\;\;\;\;\; \forall t
$$
$$
x_{m} \in {0,1} \;\;\;\;\;\;\;\; \forall m
$$

In [4]:
model = ConcreteModel()

model.t = RangeSet(6)
model.m = RangeSet(len(reach_df))

#Parameters
def ad_cost(model, m):
    return reach_df.loc[m-1,"costs"]
model.c = Param(model.m, initialize=ad_cost, within=NonNegativeIntegers)

def Ntm(model, t, m):
    clmn = "Audience type " + str(t)
    return reach_df.loc[m-1,clmn]
model.N = Param(model.t, model.m, initialize=Ntm, within=Binary)

#Variables
model.x = Var(model.m, within=Binary)

def coverage(model, t):
    return sum(model.N[t,m]*model.x[m] for m in model.m) >= 1
model.c1 = Constraint(model.t, rule=coverage)

def rule_OF(model):
    return sum(model.c[m]*model.x[m] for m in model.m)
model.objective = Objective(rule=rule_OF, sense=minimize)

opt = SolverFactory('glpk')
results = opt.solve(model)

In [5]:
model.x.pprint()

x : Size=8, Index=m
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary
      3 :     0 :   0.0 :     1 : False : False : Binary
      4 :     0 :   1.0 :     1 : False : False : Binary
      5 :     0 :   1.0 :     1 : False : False : Binary
      6 :     0 :   0.0 :     1 : False : False : Binary
      7 :     0 :   1.0 :     1 : False : False : Binary
      8 :     0 :   0.0 :     1 : False : False : Binary


# Adding Logical Conditions
Consider a condition like this:<br>
If any billboard advertisement is included in the campaign, at least one type of TV commercial should also be included. And if any TV commercial type is included in the advertisement campaign, one or both of the billboard options shold also be included. These conditions can be formulated as follows:<br>
$$
x_{TV late night} + x_{TV prime time} \ge x_{Billboard train}
$$
$$
x_{TV late night} + x_{TV prime time} \ge x_{Billboard bus}
$$
$$
x_{Billboard train} + x_{Billboard bus} \ge x_{TV late night}
$$
$$
x_{Billboard train} + x_{Billboard bus} \ge x_{TV prime time}
$$

In [7]:
del model

model = ConcreteModel()

model.t = RangeSet(6)
model.m = RangeSet(len(reach_df))

#Parameters
def ad_cost(model, m):
    return reach_df.loc[m-1,"costs"]
model.c = Param(model.m, initialize=ad_cost, within=NonNegativeIntegers)

def Ntm(model, t, m):
    clmn = "Audience type " + str(t)
    return reach_df.loc[m-1,clmn]
model.N = Param(model.t, model.m, initialize=Ntm, within=Binary)

#Variables
model.x = Var(model.m, within=Binary)

def coverage(model, t):
    return sum(model.N[t,m]*model.x[m] for m in model.m) >= 1
model.c1 = Constraint(model.t, rule=coverage)

def TV_BB(model):
    return model.x[2] + model.x[3] >= model.x[4]
model.c3 = Constraint(rule=TV_BB)

def TV_BB2(model):
    return model.x[2] + model.x[3] >= model.x[5]
model.c4 = Constraint(rule=TV_BB2)

def TV_BB3(model):
    return model.x[4] + model.x[5] >= model.x[2]
model.c5 = Constraint(rule=TV_BB3)

def TV_BB4(model):
    return model.x[4] + model.x[5] >= model.x[3]
model.c6 = Constraint(rule=TV_BB4)

def rule_OF(model):
    return sum(model.c[m]*model.x[m] for m in model.m)
model.objective = Objective(rule=rule_OF, sense=minimize)

opt = SolverFactory('glpk')
results = opt.solve(model)

In [8]:
print("Total cost of the campaign is: ", value(model.objective))
model.x.pprint()

Total cost of the campaign is:  162500.0
x : Size=8, Index=m
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary
      3 :     0 :   1.0 :     1 : False : False : Binary
      4 :     0 :   0.0 :     1 : False : False : Binary
      5 :     0 :   1.0 :     1 : False : False : Binary
      6 :     0 :   0.0 :     1 : False : False : Binary
      7 :     0 :   0.0 :     1 : False : False : Binary
      8 :     0 :   1.0 :     1 : False : False : Binary


# Covering Audience Types More Than Once
Now we can imagine a scenario where we want to cover some audience groups more than once. Let's say we want two have at least 3 audience groups that are covered more than once in the advertisement campaign. In order to account for this new condition, we need to add a separate binary variable $y_{t}$. The value of $y_{t}$ is equal to 1 when the corresponding audience group is covered at least twice in the advetisement campaign. The following formulation adds this new condition to the problem:
$$
2y_{t} \le \sum_{m}^{} N_{tm} x_{m}\;\;\;\;\;\; \forall t
$$
$$
\sum_{t}^{} y_{t} \ge 3
$$
$$
y_{t} \in {0,1}
$$

In [10]:
del model

model = ConcreteModel()

model.t = RangeSet(6)
model.m = RangeSet(len(reach_df))

#Parameters
def ad_cost(model, m):
    return reach_df.loc[m-1,"costs"]
model.c = Param(model.m, initialize=ad_cost, within=NonNegativeIntegers)

def Ntm(model, t, m):
    clmn = "Audience type " + str(t)
    return reach_df.loc[m-1,clmn]
model.N = Param(model.t, model.m, initialize=Ntm, within=Binary)

#Variables
model.x = Var(model.m, within=Binary)
model.y = Var(model.t, within=Binary)

def coverage(model, t):
    return sum(model.N[t,m]*model.x[m] for m in model.m) >= 1
model.c1 = Constraint(model.t, rule=coverage)

def form_y(model, t):
    return 2*model.y[t] <= sum(model.N[t,m]*model.x[m] for m in model.m)
model.c2 = Constraint(model.t, rule=form_y)

def three_groups(model):
    return sum(model.y[t] for t in model.t) >= 3
model.c3 = Constraint(rule=three_groups)

def rule_OF(model):
    return sum(model.c[m]*model.x[m] for m in model.m)
model.objective = Objective(rule=rule_OF, sense=minimize)

opt = SolverFactory('glpk')
results = opt.solve(model)

In [11]:
print("Total cost of the campaign is: ", value(model.objective))
model.x.pprint()

Total cost of the campaign is:  205000.0
x : Size=8, Index=m
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   1.0 :     1 : False : False : Binary
      3 :     0 :   0.0 :     1 : False : False : Binary
      4 :     0 :   1.0 :     1 : False : False : Binary
      5 :     0 :   1.0 :     1 : False : False : Binary
      6 :     0 :   0.0 :     1 : False : False : Binary
      7 :     0 :   1.0 :     1 : False : False : Binary
      8 :     0 :   0.0 :     1 : False : False : Binary
